## 3.Data Preparation

On the data preparation stage the objective is prepare the data for the modeling stage. To accoplish this objective, we will utilise the tasks raised by the preceding stage, the data analysis stage. At the end of the third stage we will have the data tables to proceed with the modeling. 

### A)Selecting the data

In [1]:
from dask.distributed import Client
import dask.dataframe as dd
import logging
import numpy as np

client = Client(processes= True, n_workers=12, threads_per_worker=2, memory_limit='20GB', silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 223.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37735,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 223.52 GiB
Comm: tcp://127.0.0.1:37539,Total threads: 2
Dashboard: http://127.0.0.1:42595/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:38973,


Let's start loading the raw data given by the Kaggle competition website.

In [2]:
blocksize = '128MB'
#loading the data
x_test = dd.read_csv("../data/originalSet/test_data.csv", blocksize=blocksize)
x_train = dd.read_csv("../data/originalSet/train_data.csv", blocksize=blocksize)
y_train = dd.read_csv("../data/originalSet/train_labels.csv", blocksize=blocksize)

In [3]:
x_test = x_test.loc[:130000]

### B) Padding the Date

In [4]:
x_test = x_test.persist()

We have a window of one year +1 month of date, but some customers_id has less then 13 months. To standardize the date, we will add the missing months on all the customers. To accomplish the task, we will first locate the missing dates, but first we have to truncate the date by month and get unique vals. 

On the Data analysis stage, we observed that the test set has 18 months, this can be a problem, let's try to truncate the extra months by put the in a cycle, with the months pass the last date, then the month goes back to the first month.

In [5]:
#initial procedure for further processes
#reset index, truncate date time by month, get unique customers ids and unique dates
def initial_procedure(df):
    df = df.reset_index(drop=True)
    #transform the date in the proper type.
    df["S_2"] = df["S_2"].astype("datetime64[ns]")
    #truncate by month
    df['S_2'] = df["S_2"].dt.strftime('%Y-%m')
    #get unique dates
    x_date = df['S_2'].unique().compute()
    x_customer = df['customer_ID'].value_counts().compute()
    return x_date, x_customer, df


In [5]:
x_train_date, x_train_customer, x_train = initial_procedure(x_train)

For the test dataset, we will truncate the extra months to 2018, in this manner we have a window of 13 months for both datasets.

In [6]:
x_test_date, x_test_customer, x_test = initial_procedure(x_test)

In [7]:
x_test_date

0     2019-02
1     2019-03
2     2019-04
3     2019-05
4     2019-06
5     2019-07
6     2019-08
7     2019-09
8     2019-10
9     2018-04
10    2018-05
11    2018-06
12    2018-07
13    2018-08
14    2018-09
15    2018-10
16    2018-11
17    2018-12
18    2019-01
Name: S_2, dtype: string

In [8]:
dates = x_test_date[3:9].astype('datetime64[ns]').dt.strftime('%Y-%m')
dates

3    2019-05
4    2019-06
5    2019-07
6    2019-08
7    2019-09
8    2019-10
Name: S_2, dtype: object

In [9]:
x_test_date = x_test_date.drop(index= dates.index)

Those dates will be changed to 2018 instead.

In [10]:
replace_dict = {date: date.replace('2019', '2018') for date in list(dates)}

In [11]:
replace_dict

{'2019-05': '2018-05',
 '2019-06': '2018-06',
 '2019-07': '2018-07',
 '2019-08': '2018-08',
 '2019-09': '2018-09',
 '2019-10': '2018-10'}

In [12]:
x_test = x_test.replace(to_replace = replace_dict)

In [14]:
x_test_date

0     2019-02
1     2019-03
2     2019-04
9     2018-04
10    2018-05
11    2018-06
12    2018-07
13    2018-08
14    2018-09
15    2018-10
16    2018-11
17    2018-12
18    2019-01
Name: S_2, dtype: string

We have to apply the initial procedure again to take the new dates.

In [17]:
#get missing customers dates. parameters: dataframe, ocorrences of custumers id from initial procedure.
#return customer and list of dates of the customers who has missing dates. Dataframe type.
def get_missing_customer(df, x_customer, len_date):
    #customers with missing data
    cust_wmissing = x_customer[x_customer < len_date].index
    #selecting customers with missing on the database, we only need the customer_ID and the S_2
    x_missing = df[df['customer_ID'].isin(cust_wmissing)][['customer_ID', 'S_2']]
    #aggregate the customers by id and adding the dates to a list. 
    x_grouped = x_missing.groupby(['customer_ID'])[['S_2']].aggregate([list]).compute()
    return x_grouped 

Getting missing customers with a list of present dates. The list them will compared with all dates that has to be present to check what date is not present.

In [14]:
x_train_grouped = get_missing_customer(x_train, x_train_customer, len(x_train_date))

In [15]:
x_train_grouped 

,S_2
,list
customer_ID,
0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba17...,"['2018-01', '2018-02', '2018-03']"
000391f219520dbca6c3c1c46e0fab569da163f79ee266...,"['2017-12', '2018-01', '2018-02', '2018-03']"
0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,"['2017-07', '2017-08', '2017-09', '2017-10', '..."
00057c2d8d887fa3f777d97dc939700731575772e6c990...,"['2017-12', '2018-01', '2018-02', '2018-03']"
0005a6ae24fd274640a237ea56c43b1ef9e32077ad168a...,"['2017-03', '2017-04', '2017-06', '2017-07', '..."
...,...
fffe13e28dc3ceadf28249b596ba25df93e38ec53d38cf...,"['2018-01', '2018-02', '2018-03']"
fffe2bc02423407e33a607660caeed076d713d8a5ad323...,"['2017-08', '2017-09', '2017-10', '2017-11', '..."


There is 72879 incomplete customers. 

In [18]:
x_test_grouped = get_missing_customer(x_test, x_test_customer,len(x_test_date))

Same procedure to the test set.

In [19]:
x_test_grouped

,S_2
,list
customer_ID,
00000469ba478561f23a92a868bd366de6f6527a684c9a...,"['2019-02', '2019-03', '2019-04', '2018-05', '..."
00018b499469c9f10a1b428cbcf7419efa4b3336e6dd78...,"['2018-11', '2018-12', '2019-01', '2019-02', '..."
0002b4b7a4f5b165bc71794ba473cc5bfa2267c7bf8f6e...,"['2019-04', '2018-05', '2018-06', '2018-07', '..."
000311b99b226f7bf3a032671edb917f8a4edf98ab06ac...,"['2018-11', '2018-12', '2019-01', '2019-02', '..."
00034153f0d5a7e474b05799e9ca6d30fa7008065b1958...,"['2018-07', '2018-08', '2018-09', '2018-10', '..."
...,...
fffbcd8f05f77ac346c24aa8a9ab255a05553f0bfab1e3...,"['2019-03', '2019-04']"
fffce03d67327288677c023f5cf78fecec19a67db8eb0a...,"['2019-01', '2019-02', '2019-03', '2019-04']"


On the test data set we have 113292 customers who needs new date rows.

Now, we can procedd padding them. First, we will find the difference between all the dates and the current dates.

In [20]:
from ast import literal_eval
#Parameters:df_grouped and total dates for the dataset, given by the function inicial_procedure and 
#get_missing_customer.
#for each customer, identify the missing date.
def get_s2_diff(df_grouped, total_date):
    #for each customer missing customer, identify the missing date.
    new_columns = []
    for df_g in df_grouped.iterrows():
        #the difference between the list of all dates and the customer date is what we want.
        new_columns.append(np.setdiff1d(total_date.to_numpy(), np.array(literal_eval(df_g[1]['S_2']['list']))))
    return new_columns

In [19]:
x_train_grouped['s2-diff'] = get_s2_diff(x_train_grouped, x_train_date)
#example
print(f"dates present on the row:{x_train_grouped['S_2']['list'][0]}, dates missing:{x_train_grouped['s2-diff'][0]}")

dates present on the row:['2018-01', '2018-02', '2018-03'], dates missing:['2017-03' '2017-04' '2017-05' '2017-06' '2017-07' '2017-08' '2017-09'
 '2017-10' '2017-11' '2017-12']


Same for the test set.

In [22]:
x_test_grouped['s2-diff'] = get_s2_diff(x_test_grouped, x_test_date)

Now for each date in s2-diff, we create a new row. 

In [23]:
import numpy as np
import pandas as pd
#pass the customer_ID, date_missing, and columns
#return the new rows on a new dataframe to add on the old one
def get_pad_date(customer, date_missing, columns):
    new_row = {column:[] for column in columns}
    #for each date missing, add on the dictionary
    for date in date_missing:
        new_row['customer_ID'].append(customer)
        new_row['S_2'].append(date)
        #fill the rest with nulls
        for column in columns[2:]:
            new_row[column].append(np.nan)
    #transform dict on new dataframe
    new_df = pd.DataFrame(new_row, columns=columns)
    return new_df

In [24]:

#build missing to add to dataset
def build_missing(df, x_grouped):
    columns = df.columns[:]
    new_rows_list = []
    #for each customer, create the rows missing.
    for customer, date in  x_grouped.iterrows():
        new_rows_list.append(get_pad_date(customer, date['s2-diff'][0], columns))
    
    #New rows formatted on a dataFrame
    new_rows_df = pd.concat(new_rows_list)
    print(f'There will be added {len(new_rows_df)} new rows to the train dataset.')
    df = dd.multi.concat([df, new_rows_df])
    
    #setting a new index to sort the customers together
    df['new_index'] = df['customer_ID']
    df['new_index'] += df['S_2']
    
    #new index is the customer id concatenated with date(dask do not accept multiindex)
    df = df.set_index(['new_index'], sort=True)
    return df
    

In [19]:
x_train = x_train.persist()
x_test = x_test.persist()

In [23]:
len(x_train)

5531451

In [24]:
x_train = build_missing(x_train, x_train_grouped)

There will be added 434418 new rows to the train dataset.


/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 11.16 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [25]:
len(x_train)

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 10.99 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


5965869

In [25]:
x_test = build_missing(x_test, x_test_grouped)

There will be added 662303 new rows to the train dataset.


/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 17.22 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [26]:
cust_counts = x_test.customer_ID.value_counts().compute()

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 16.87 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [27]:
cust_counts

customer_ID
000c5539d1125ff38a83172a3a4e5cb14a9c338090df1c...    14
0022bd62eb7f07460a6d550791ed8885226ce8e5d8322d...    14
0028e742f606e415153fae93ad58f15484505ba3cb26df...    14
0031546c158263c8ad2075d6b7cf0a4b9decc923e9f994...    14
0048d7503cba77761253f89b8380bd08fe799cb0568207...    14
                                                     ..
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...    13
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...    13
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...    13
ffffddef1fc3643ea179c93245b68dca0f36941cd83977...    13
fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...    13
Name: count, Length: 924621, dtype: int64[pyarrow]

In [28]:
extra_cust = cust_counts[cust_counts==14]

In [29]:
extra_cust

customer_ID
000c5539d1125ff38a83172a3a4e5cb14a9c338090df1c...    14
0022bd62eb7f07460a6d550791ed8885226ce8e5d8322d...    14
0028e742f606e415153fae93ad58f15484505ba3cb26df...    14
0031546c158263c8ad2075d6b7cf0a4b9decc923e9f994...    14
0048d7503cba77761253f89b8380bd08fe799cb0568207...    14
                                                     ..
ffd5f8fc58a81639970fb2b0bfb367d17f15f2654ab7b3...    14
ffdded2b8530fd912cb3c37c91012c5382a6d7e6c6b1de...    14
ffdeb508e1907f26628c452ab13c204ef882613cbbb2e6...    14
ffe15797052587a5ec3a5f6228edd0e2c212abb6118168...    14
ffe45d42db82aa5a30d4b185030b09e3d3952203786493...    14
Name: count, Length: 5992, dtype: int64[pyarrow]

We gonna save this custumers on a pandas dataframe because dask dataframe doesn't handle duplicates so well and is just 5992 clients. After the handling, the custumers will be incorporated again on the dataset.

In [33]:
dupli_customers = x_test[x_test['customer_ID'].isin(extra_cust.index)].compute()

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 17.26 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [35]:
safet_car = dupli_customers.copy()

In [58]:
dupli_customers = dupli_customers.reset_index()

In [62]:
dupli_customers = dupli_customers.drop_duplicates(subset='new_index', keep='first').set_index('new_index')

In [64]:
index_drop = safet_car.index

In [68]:
x_test = x_test[~x_test['customer_ID'].isin(extra_cust.index)]

In [71]:
dupli_dask = dd.from_pandas(dupli_customers, npartitions=1)

In [72]:
x_test = dd.multi.concat([x_test, dupli_customers], axis='index')

Checkpoint

In [73]:
#x_train.to_parquet('../data/train/')
x_test.to_parquet('../data/test/')

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 29.52 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [74]:
x_test = dd.read_parquet('../data/test/')

In [75]:
cust_counts = x_test.customer_ID.value_counts().compute()

In [76]:
cust_counts

customer_ID
00000469ba478561f23a92a868bd366de6f6527a684c9a...    13
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...    13
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...    13
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...    13
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...    13
                                                     ..
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...    13
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...    13
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...    13
ffffddef1fc3643ea179c93245b68dca0f36941cd83977...    13
fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...    13
Name: count, Length: 924621, dtype: int64[pyarrow]

In [22]:
x_train.to_parquet("../data/train/")

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 11.03 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [81]:
x_train = dd.read_parquet('../data/train/')
x_test = dd.read_parquet('../data/test/')

In [25]:
len(x_train)

5965869

In [78]:
len(x_test)

12020073

### B)Missing Values

For the missing values, we will use two methods. First we will apply the last valid, but last valid doesn't fill the spaces where there is no last valid. So next we will fill with the next valid. We will follow this order because maybe if the value is missing is because the last value is still valid. We are considering that the value doesn't have changed. 

Note: The method is filling spaces that is not on the same customer id, there's room for improvment apply the same procedure selecting the client before making changes. the values can mislead the model, filling the gaps with incosistent values. 

Moreover, we notice from data analysis that some columns has little samples. We think that this features maybe is important but if we apply the procedure described above, the colum will be full of misleading information. To solve the problem we will set a threshold. If the columns has elements to surpass the threshold, the procedure above will be applied, if not the column will be dropped for now. 

In [79]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300 

In [82]:
len(x_train)

5965869

Taking in to account that we added a round 500.000 empty rows on the previous step, and we have 5.965.869 of rows now, we gonna use a threshold of  2000000, which corresponds to 1500000 of the original empty spaces.

Before this procedure, some columns already has a value for NAN, -1, so we gonna use this information too and in those situations we gonna remove the columns and just fill with -1 on this columns. 

In [83]:
categorical_feat =['B_30', 'B_38', 'D_114', 'D_116', 
                   'D_117', 'D_120', 'D_126', 'D_63',
                   'D_64', 'D_66', 'D_68']
x_train[categorical_feat].info(verbose=True)

<class 'dask.dataframe.core.DataFrame'>
Index: 5965869 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a2017-03 to fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab046812018-03
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   B_30    5529435 non-null      float64
 1   B_38    5529435 non-null      float64
 2   D_114   5354735 non-null      float64
 3   D_116   5354735 non-null      float64
 4   D_117   5354735 non-null      float64
 5   D_120   5354735 non-null      float64
 6   D_126   5414635 non-null      float64
 7   D_63    5531451 non-null      string
 8   D_64    5314009 non-null      string
 9   D_66    623354 non-null      float64
10   D_68    5314948 non-null      float64
dtypes: float64(9), string(2)
memory usage: 898.9 MB


The type of two columns is object, let's take a closer look on what's inside the categorical feature.

In [8]:
for cat_feat in categorical_feat:
    print(f'Feature {cat_feat}, unique values: {x_train[cat_feat].unique().compute()}')


Feature B_30, unique values: 0    0.0
1    2.0
2    1.0
3    NaN
Name: B_30, dtype: float64
Feature B_38, unique values: 0    2.0
1    1.0
2    3.0
3    5.0
4    6.0
5    7.0
6    NaN
7    4.0
Name: B_38, dtype: float64
Feature D_114, unique values: 0    1.0
1    0.0
2    NaN
Name: D_114, dtype: float64
Feature D_116, unique values: 0    0.0
1    NaN
2    1.0
Name: D_116, dtype: float64
Feature D_117, unique values: 0    4.0
1   -1.0
2    6.0
3    2.0
4    1.0
5    NaN
6    3.0
7    5.0
Name: D_117, dtype: float64
Feature D_120, unique values: 0    0.0
1    1.0
2    NaN
Name: D_120, dtype: float64
Feature D_126, unique values: 0    1.0
1    NaN
2    0.0
3   -1.0
Name: D_126, dtype: float64
Feature D_63, unique values: 0      CR
1      CO
2    <NA>
3      CL
4      XZ
5      XM
6      XL
Name: D_63, dtype: string
Feature D_64, unique values: 0       O
1       R
2    <NA>
3       U
4      -1
Name: D_64, dtype: string
Feature D_66, unique values: 0    NaN
1    1.0
2    0.0
Name: D_66, dty

Observing the features we notice five distict cases, and requires three distict handling, the cases are:

1. The columns B_30, B_38, D_68 is a type of ordinal encoder, which means the columns has ordered integers coding the categories. Those columns doesn't have the value -1 for NAN, them we gonna replace missing values(NAN) on those columns with -1. 
2. The columns D_114, D_116, D_120, D_66 is a encoded on a binary format. For those columns we gonna use the plots of the stage 2.Data Analysis, we will put the missing elements on the class that has more impurity. We will drop those class on future procedures, then only the relevant variable will remain at the end of data preparation stage.*
3. The columns D_117, D_126, already has the -1 for NAN. We gonna put the NAN on the -1 class.
4. The column D_63 is filled with string, to apply the next steps we will need that all values to be of type string, so we gonna replace NAN to a string 'NAN'. When we encode this column we will drop the column who accounts for this value.
5. The column D_64 has mixed types filling the rows. We need to put the NAN on the -1 categorie, however we also need to transform -1 to some string, we will choose the string 'NAN' as well.

*On further analysis of the case 2, for D_114, D_116, D_120,  there is no class to put missing that will not interfer, them a new class of value -1 will be created, after the one-hot the class will be dropped from the columns. For D_66, we can put the missing to the class 0, and them drop 0 on next stage.

Now we gonna proccedd with the preprocessing.

In [84]:
#apply the pipeline for process missing values. 
def set_missing_pipeline(df):
    #fill NAN with -1.
    df[['B_30', 'B_38', 'D_68', 'D_114',
               'D_116', 'D_120', 'D_117', 'D_126']] = df[['B_30', 'B_38', 'D_68', 
                                                           'D_114', 'D_116', 'D_120', 
                                                           'D_117', 'D_126']].fillna(value=-1)

    #replace with 0
    df[['D_66']] = df[['D_66']].fillna(value=0)
    
    #fill NAN with '-1'
    df[['D_63', 'D_64']] = df[['D_63', 'D_64']].fillna(value='-1')
    #replace -1 with string
    df[['D_63', 'D_64']] = df[['D_63', 'D_64']].replace(to_replace = '-1' , value='NAN')

    return df


In [85]:
x_train = set_missing_pipeline(x_train)

In [86]:
x_test = set_missing_pipeline(x_test)

The status quo of missing is as follow:

In [87]:
x_train[categorical_feat].info(verbose=True)

<class 'dask.dataframe.core.DataFrame'>
Index: 5965869 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a2017-03 to fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab046812018-03
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   B_30    5965869 non-null      float64
 1   B_38    5965869 non-null      float64
 2   D_114   5965869 non-null      float64
 3   D_116   5965869 non-null      float64
 4   D_117   5965869 non-null      float64
 5   D_120   5965869 non-null      float64
 6   D_126   5965869 non-null      float64
 7   D_63    5965869 non-null      string
 8   D_64    5965869 non-null      string
 9   D_66    5965869 non-null      float64
10   D_68    5965869 non-null      float64
dtypes: float64(9), string(2)
memory usage: 900.7 MB


In [88]:
# Count total NaN at each column in a DataFrame
nan_counts = x_train.isnull().sum().compute()
print(" \nCount total NaN at each column x_train: \n\n", nan_counts.sort_values())

 
Count total NaN at each column x_train: 

 customer_ID          0
D_116                0
D_64                 0
D_117                0
D_120                0
D_66                 0
D_68                 0
D_126                0
B_38                 0
D_63                 0
B_30                 0
D_114                0
S_2                  0
B_32            434418
P_4             434418
B_23            434418
S_15            434418
D_71            434418
S_13            434418
R_6             434418
S_12            434418
D_92            434418
D_93            434418
D_94            434418
B_18            434418
R_24            434418
D_65            434418
R_25            434418
D_96            434418
D_75            434418
R_23            434418
B_24            434418
B_36            434418
R_19            434418
S_19            434418
B_31            434418
R_18            434418
R_17            434418
D_86            434418
S_18            434418
R_21            434418
R_16        

In [89]:
len(nan_counts[nan_counts < 1000000])

152

columns to drop:

In [90]:
columns_drop = list(nan_counts[nan_counts > 1000000].index)
columns_drop

['S_3',
 'D_42',
 'D_43',
 'D_46',
 'D_48',
 'D_49',
 'D_50',
 'D_53',
 'S_7',
 'D_56',
 'S_9',
 'D_61',
 'D_62',
 'B_17',
 'D_73',
 'D_76',
 'D_77',
 'R_9',
 'D_82',
 'B_29',
 'D_87',
 'D_88',
 'D_105',
 'D_106',
 'R_26',
 'D_108',
 'D_110',
 'D_111',
 'B_39',
 'S_27',
 'B_42',
 'D_132',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_142']

In [91]:
x_train = x_train.drop(labels = columns_drop, axis = 1)
x_test = x_test.drop(labels = columns_drop, axis = 1)

In [92]:
features = x_train.columns[2:]

In [93]:
#set missing features, applying first the last valid them next value.
def set_missing_feat(df, features):
    #for each feature, fill nan
    for feat in features:
        #last valid
        df[feat] = df[feat].fillna(method='ffill')
        #next valid
        df[feat] = df[feat].fillna(method='backfill')
    return df

In [94]:
x_train = set_missing_feat(x_train, features)
x_test = set_missing_feat(x_test, features)

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/core.py:1689: PerformanceWarning: Falling back on a non-pyarrow code path which may decrease performance.
  meta = self._meta_nonempty.fillna(
/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/core.py:1721: PerformanceWarning: Falling back on a non-pyarrow code path which may decrease performance.
  meta = self._meta_nonempty.ffill(limit=limit, axis=axis)
/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/core.py:1689: PerformanceWarning: Falling back on a non-pyarrow code path which may decrease performance.
  meta = self._meta_nonempty.fillna(
/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/core.py:1735: PerformanceWarning: Falling back on a non-pyarrow code path which may decrease performance.
  meta = self._meta_nonempty.bfill(limit=limit, axis=axis)
/home/codemaster/anaconda3/envs/

In [95]:
x_train.to_parquet('../data/train-checkpoint/')
x_test.to_parquet('../data/test-checkpoint/')

All fields were full filled. We can go to the next step.

In [96]:
x_train = dd.read_parquet('../data/train-checkpoint/')
x_test = dd.read_parquet('../data/test-checkpoint/')

In [80]:
x_train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,D_41,B_3,D_44,...,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145
new_index,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03,0.938469,0.001733,0.008724,1.006838,0.009228,0.008771,0.004709,0.000630,...,0.002052,0.005972,0.004345,0.001535,0.002427,0.003706,0.003818,0.000569,0.000610,0.002674
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04,0.936665,0.005775,0.004923,1.000653,0.006151,0.000798,0.002714,0.002526,...,0.001034,0.004838,0.007495,0.004931,0.003954,0.003167,0.005032,0.009576,0.005492,0.009217
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05,0.954180,0.091505,0.021655,1.009672,0.006815,0.007598,0.009423,0.007605,...,0.005681,0.005497,0.009227,0.009123,0.003269,0.007329,0.000427,0.003429,0.006986,0.002603
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06,0.960384,0.002455,0.013683,1.002700,0.001373,0.000685,0.005531,0.006406,...,0.007108,0.008261,0.007206,0.002409,0.006117,0.004516,0.003200,0.008419,0.006527,0.009600
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07,0.947248,0.002483,0.015193,1.000727,0.007605,0.004653,0.009312,0.007731,...,0.009680,0.004848,0.006312,0.004462,0.003671,0.004946,0.008889,0.001670,0.008126,0.009827


### C)Transform data

The next step is transform the data. Data analysis pointed out scaling and categorical value needs handling. Begining with categorical features, some of them has strings on it and need encoding. To solve the problem we will keep the things simple and just use one-hot encoding. The one-hot encoding will create one extra column for each categorie, the new column will be of a binary type, 1 representing the categorie and 0 representing not on the categorie. The drawback of one-hot encoding is that they increase the dimensionality of the data. In this case we already had 152 features, although we have many samples, dimensionality is still a problem because each feature increases the dimension and with that we have few sample cases for a given set of features, making learning difficult. Maybe transformation on the feature space will be necessary, for the first version we will ignore this issue. 

Procceding with the one-hot encoding, first we will take a look on the categorical features.

Some categorical features, has ordened values. Maybe there is a good motivation for that because some categorical features may have a implicity order that may help in the task of predicting the default, i.e. schorlarity level. Under those circumstances, we will exclude those features from the one hot encoder. Moreover, there's some features that has just 0 or 1 as categories, those features doesn't need preprocessing, they are already encoded. Also, this will help reduce the dimensionality because some of the excluded features has seven categories, which means that if we applied one-hot procedure on this categorie, it will lead to seven more columns. On total, avoiding one hot encoding on those categorical features we have save 35 new columns, the only columns how needs encoding is D_63, D_64 and some features we had filled with -1, D_114, D_116, D_120.

Now applying the one-hot encoding.

In [97]:
from dask_ml.preprocessing import OneHotEncoder
onehot_features = ['D_63', 'D_64', 'D_114', 'D_116', 'D_120']
x = x_train[onehot_features].astype('category')
#dropping the first categorie, for dimensionality reduction.
one = OneHotEncoder(sparse_output=False)
one = one.fit(x.categorize())

In [98]:
x_train[onehot_features].head()

,D_63,D_64,D_114,D_116,D_120
new_index,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,CR,O,1.0,0.0,0.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,CR,O,1.0,0.0,0.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,CR,O,1.0,0.0,0.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,CR,O,1.0,0.0,0.0
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,CR,O,1.0,0.0,0.0


In [99]:
x_train = dd.multi.concat([x_train, one.transform(x_train[onehot_features])],axis=1)

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [100]:
x_test = dd.multi.concat([x_test, one.transform(x_test[onehot_features])],axis=1)

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


Now we can drop the original features encoded and the missing ones.

In [101]:
columns_drop = ['D_64', 'D_63', 'D_120','D_116','D_114','D_64_NAN', 'D_63_NAN', 'D_120_-1.0','D_116_-1.0','D_114_-1.0']
x_train = x_train.drop(labels = columns_drop, axis = 1)
x_test = x_test.drop(labels = columns_drop, axis = 1)

Some features has a max of 8 or of 3 and others a max of 1. This variance of range can change the objective function of the final model. Then, we will stardardize using robust scaler. The selected method was arbitrary and can be modified for future versions, is just a common choice.

In [102]:
features = x_train.columns[2:]

In [103]:
for feat in features:
    x_train[feat] = x_train[feat].astype('float64')  

In [104]:
from dask_ml.preprocessing import RobustScaler
rb = RobustScaler(unit_variance= True)
rb = rb.fit(x_train[features])

In [105]:
x_train = dd.multi.concat([x_train[x_train.columns[:2]], rb.transform(x_train[features])],axis=1)
x_test = dd.multi.concat([x_test[x_test.columns[:2]], rb.transform(x_test[features])],axis=1)

/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(
/home/codemaster/anaconda3/envs/america-exp/lib/python3.11/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


Before ending this section, we will save the scaler, normalizer, the encoder and the new dataset.

In [106]:
x_train.to_parquet('../data/V1Set/train/')
x_test.to_parquet('../data/V1Set/test/')

In [102]:
x_train.describe().compute()

,P_2,D_39,B_1,B_2,R_1,D_41,B_3,D_44,B_4,D_45,...,D_63_XZ,D_64_O,D_64_R,D_64_U,D_114_0.0,D_114_1.0,D_116_0.0,D_116_1.0,D_120_0.0,D_120_1.0
count,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,...,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06,5.965869e+06
mean,-1.919554e-01,5.501743e-01,5.341971e-01,-2.271399e-01,1.282582e+01,9.217330e+00,4.916127e-01,4.537652e-01,3.215359e-01,9.546490e-02,...,4.322254e-03,-5.116815e-01,1.408197e-01,2.553606e-01,3.416530e-01,-4.440914e-01,-1.035490e-01,1.110651e-03,-2.072030e-01,1.047646e-01
std,6.649246e-01,1.057988e+00,1.335595e+00,4.673694e-01,3.953382e+01,3.469073e+01,1.004390e+00,9.081377e-01,9.388477e-01,7.063161e-01,...,6.560162e-02,4.998636e-01,3.478355e-01,4.360638e-01,4.742639e-01,4.968644e-01,3.046747e-01,3.330793e-02,4.053023e-01,3.062499e-01
min,-3.194734e+00,-3.571759e-02,-4.772506e+01,-9.473637e-01,-1.019519e+00,-1.013872e+00,-7.727196e-02,-3.392805e-02,-4.068380e-01,-6.366076e-01,...,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00
25%,-5.884385e-01,-1.695727e-02,-1.872797e-01,-7.815938e-01,-4.941428e-01,-5.096604e-01,-5.051682e-02,-1.650117e-02,-2.671301e-01,-4.442957e-01,...,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.115615e-01,9.830427e-01,8.127203e-01,2.184062e-01,5.058572e-01,4.903396e-01,9.494832e-01,9.834988e-01,7.328699e-01,5.557043e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,7.664392e-01,2.016755e+01,8.036171e+00,2.265368e-01,5.576164e+02,1.466562e+03,6.828378e+00,2.283863e+01,8.267809e+01,4.052452e+00,...,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00


In [107]:
import pickle

with open("../var/hotencoder", "wb") as f:
    pickle.dump(one, f)
with open("../var/scaler", "wb") as f:
    pickle.dump(rb,f)